# Read an input JSON statement

> The methods in this notebook implement the functionalities for reading the data after exporting them from Learning Locker. 
Statements are usually provided in the form of ```csv``` files, but in case you want to check all the metadata of a statement, you can directly work with the ```json``` version of a statement.
Here we show how to import the data, parse the json or csv file, and to filter or remove data which are not necessary. 

In [ ]:
#| default_exp input_json

In [ ]:
#| hide
from nbdev.showdoc import *

The libraries used to import the data:

In [ ]:
import json
import pandas
from typing import Union, List
from datetime import datetime
from pathlib import Path
from fastcore.test import *

As an example, in this package we provide two files, a ```csv``` containing a collection of about 1000 statements, as well as a ```json``` of a stetement and all its related metadata

In [ ]:
csv_file = '../example_statements.csv'
json_file = '../example_single_statement.json'

Let's start parsing the json file

In [ ]:
#| export
def load_statement(json_file: str # Filename of the json containing the statement
                  ) -> dict: # A dictionary representing the statement structure
    """
    Load a json from file and store the information in a Python dictionary object.
    If the file does not exist, returns an empty dict and print an error message
    """
    if Path(json_file).exists():
        with open(json_file) as f:
            return json.load(f)
    else:
        print("ERROR: The specified file does not exist")
        return dict()

In [ ]:
my_statement = load_statement(json_file)

In [ ]:
#| export
def pretty_print_statement(statement: dict, # the statement dict imported from JSON
                          indent: int=4 # indentation used when printing
                          ) -> None:
    """
    Displays the content of the statement in a human readable format
    """
    print(json.dumps(statement, indent=indent))

In [ ]:
sample_json = json.loads('["foo", {"bar": ["baz", null, 1.0, 2]}]')
pretty_print_statement(sample_json, indent=2)

[
  "foo",
  {
    "bar": [
      "baz",
      null,
      1.0,
      2
    ]
  }
]


##### Extract the metadata information
The following methods are used to extract the metadata fields we may be interested in

In [ ]:
#| export
def get_value(statement: dict, # Our xAPI statement imported from JSON
              key: str # The key we are interested in
             ) -> Union[str, dict, None]: # The value associated to the key in the statement
    """
    Return the value associated to the specified key in the statement dictionary.
    If the key does not exist, returns None
    """
    if key in statement:
        return statement[key]
    else:
        return None

In [ ]:
test_eq(get_value(my_statement, "not_a_key"), None)
test_eq(get_value(my_statement, "stored"), "2022-09-30T13:34:35.959Z")

In [ ]:
#| export
def get_stored(statement: dict, # Our xAPI statement imported from JSON
              ) -> datetime: # datetime object representing the time the statement was stored in the database
    """
    Extract the date and time information of when the statement was stored in the database
    """
    stored_str = get_value(statement, "stored")
    return datetime.strptime(stored_str, "%Y-%m-%dT%H:%M:%S.%f%z")

In [ ]:
my_date = datetime.strptime("2022-09-30T13:34:35.959Z", "%Y-%m-%dT%H:%M:%S.%f%z")
test_eq(get_stored(my_statement), my_date)

In [ ]:
#| export
def get_active(statement: dict, # Our xAPI statement imported from JSON
              ) -> bool: # Boolean representive whether active or not
    """
    Extract the Active field from the statement
    """
    return get_value(statement, "active")

In [ ]:
test_eq(get_active(my_statement), True)

In [ ]:
#| export
def get_client(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # ID of the client
    """
    Extract the client field from the statement
    """
    return get_value(statement, "client")

In [ ]:
test_eq(get_client(my_statement), "60ffcf8d448b2d059a63e3c4")

In [ ]:
#| export
def get_LRS(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # ID of the Learning Record Store
    """
    Extract the Learning Record Store ID field from the statement
    """
    return get_value(statement, "lrs_id")

In [ ]:
test_eq(get_LRS(my_statement), "60ffcf8d448b2d059a63e3c3")

In [ ]:
def get_completed_fw_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed forwarding queues in the statement
    """
    Extract the List of completed forwarding queues in the statement
    """
    return get_value(statement, "completedForwardingQueue")

In [ ]:
test_eq(get_completed_fw_queues(my_statement), list())

In [ ]:
def get_failed_fw_log(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of failed forwarding log messages in the statement
    """
    Extract the List of failed forwarding log messages in the statement
    """
    return get_value(statement, "failedForwardingLog")

In [ ]:
test_eq(get_failed_fw_log(my_statement), list())

In [ ]:
def get_completed_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed queues in the statement
    """
    Extract the List of completed queues in the statement
    """
    return get_value(statement, "completedQueues")

In [ ]:
COMPL_QS = ["STATEMENT_FORWARDING_QUEUE", "STATEMENT_PERSON_QUEUE", "STATEMENT_QUERYBUILDERCACHE_QUEUE"]
test_eq(get_completed_queues(my_statement), COMPL_QS)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()